This File will make summary tables for the Wedge DB. After the tables are made, they will be sent to a local DataBase that is created in this file.

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import random
import sqlite3
import pandas as pd
from pandas import DataFrame
import pandas_gbq

You will need to provide your own service path, service file, gbq project id, and dataset id. I have redacted mine so you can add yours into the quotation marks below

In [ ]:
service_path = ""
service_file = ""
gbq_proj_id = ""
gbq_dataset_id =""

private_key = service_path + service_file

In [ ]:
# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

Here are all of the querys needed to complete this section. Note: The third query still needs some work after we get it into the data frame

In [ ]:
qry1 = """SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(HOUR FROM datetime)) AS Hour,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || Register_Num || Emp_Num || Trans_Num)) AS Transactions,
    SUM(CASE WHEN(Trans_Status = 'V' OR Trans_Status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `joynerwedge.JoynerWedgeV3.transArchive*`
    
    WHERE Card_No != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour""" 

In [ ]:
qry2 = """SELECT card_no,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || Register_Num || Emp_Num || Trans_Num)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `joynerwedge.JoynerWedgeV3.transArchive*`

    WHERE department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY card_no, Year, Month
    ORDER BY card_no, Year, Month"""

In [ ]:
qry3 = """SELECT
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    upc,
    description,
    p1.department AS dept_no,
    SUM(total) as Sales,
    COUNT(DISTINCT(Date(datetime) || register_num || emp_num || trans_num)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `joynerwedge.JoynerWedgeV3.transArchive*` AS p1
    
    LEFT OUTER JOIN `joynerwedge.JoynerWedgeV3.DepartmentLookup` AS dl ON p1.department = dl.department
    
    WHERE card_no != 3
    AND p1.department != 0
    AND p1.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
    GROUP BY Year, Month, upc, description, dept_no
    ORDER BY description, Year, Month DESC"""

Now we add each query into a Dataframe via pd.read-gbq

In [ ]:
#pandas.read_gbq(query, project_id=None, index_col=None, col_order=None, reauth=False,
#auth_local_webserver=False, dialect=None, location=None, configuration=None, credentials=None,
#use_bqstorage_api=None, max_results=None, progress_bar_type=None)
df1 = pd.read_gbq(qry1,gbq_proj_id)
df1.head(10)

In [ ]:
df2 = pd.read_gbq(qry2,gbq_proj_id)
df2.head(10)

In [ ]:
df3 = pd.read_gbq(qry3,gbq_proj_id)
df3.head(10)

Now that all of the data is in the DataFrames, lets fix the third dataframe so that we have department names


In [ ]:
#I got the inspiration for this code from  https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o
def dept_name (row):
    if row['dept_no'] == 1.0:
        return "Packaged Grocery"
    if row['dept_no'] == 2.0:
        return "Produce"
    if row['dept_no'] == 3.0:
        return "Bulk"
    if row['dept_no'] == 4.0:
        return "Ref Grocery"
    if row['dept_no'] == 5.0:
        return "Cheese"
    if row['dept_no'] == 6.0:
        return "Frozen"
    if row['dept_no'] == 7.0:
        return "Bread"
    if row['dept_no'] == 8.0:
        return "Deli"
    if row['dept_no'] == 9.0:
        return "Gen Merch"
    if row['dept_no'] == 10.0:
        return "Supplements"
    if row['dept_no'] == 11.0:
        return "Personal Care"
    if row['dept_no'] == 12.0:
        return "Herbs & Spices"
    if row['dept_no'] == 13.0:
        return "Meat"
    if row['dept_no'] == 14.0:
        return "Juice Bar"
    if row['dept_no'] == 15.0:
        return "Misc P/I"
    if row['dept_no'] == 16.0:
        return "Fish & Seafood"
    if row['dept_no'] == 17.0:
        return "Bakehouse"
    if row['dept_no'] == 18.0:
        return "Flowers"
    if row['dept_no'] == 19.0:
        return "WedgeWorldWide"
    if row['dept_no'] == 20.0:
        return "Misc P/I WWW"
    if row['dept_no'] == 21.0:
        return "Catering"
    if row['dept_no'] == 22.0:
        return "Beer & Wine"
    

In [ ]:
# This adds the column and its contents to the dataframe
df3['Dept_Name'] = df3.apply (lambda row: dept_name(row), axis=1)

In [ ]:
   df3.head(10)

Now that the Dataframe has the department names, we can send it to the database, but we need to make the database first. Making the DB and adding the first dataframe to the db will happen in the first cell. 

In [ ]:
db = sqlite3.connect('Wedge_Task3.db')
#DataFrame.to_sql(name, con, schema=None, if_exists='fail',
#index=True, index_label=None, chunksize=None, dtype=None, method=None) 
#This is the syntax for what we are doing
df1.to_sql("Output1",db)

In [ ]:
df2.to_sql("Output2",db)

In [ ]:
df3.to_sql("Output3",db)